**<h1><p style="text-align: center;">GenTax Report Automation Script: GRAS ver. 1.9</p>**

In [1]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install -U pyautoit
# !pip install pypiwin32
# !pip install --upgrade holidays

import autoit
import getpass
import time
import os 
import datetime
import win32com.client as win32
import holidays
import glob
import datetime as dt

from datetime import datetime, timedelta
from win32com.client import Dispatch
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()

options.add_argument("--headless")
browser =webdriver.Chrome()
browser = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install(), options = options))
browser.maximize_window()
browser.switch_to.window(browser.window_handles[0])

# Creating output folder

In [264]:
today = datetime.now()

os.mkdir("//Trdecomsrv/H/GRAS/" + today.strftime('%Y-%b-%d at %Hh-%Mm-%Ss'))
folder_path = "//Trdecomsrv/H/GRAS/"
files_path = os.path.join(folder_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True) 
destination_folder = (files[0])
destination_folder


'//Trdecomsrv/H/GRAS\\2023-Nov-22 at 16h-17m-30s'

# Providing login credentials

In [3]:
my_login = getpass.getpass("Please type your GenTax login id:\n")

Please type your GenTax login id:
 ········


In [4]:
my_password = getpass.getpass("Please type your GenTax password:\n")

Please type your GenTax password:
 ········


In [5]:
url = 'https://gentaxprod.trd.state.nm.us/GenTax/dPWmgh2N/#1'
browser.get(url)
browser.maximize_window()

In [6]:
time.sleep(2)
username_el = browser.find_element("name", "Dc-c")
username_el.send_keys(my_login)

In [7]:
password_el = browser.find_element("name", "Dc-d")
password_el.send_keys(my_password)

# Clicking the logon button

In [8]:
time.sleep(1.5)
submit_btn_el = browser.find_element("css selector", "[class='ButtonCaptionText']")

In [9]:
submit_btn_el.click()

# Clicking the Report button

In [10]:
time.sleep(1.5)
Report_btn_el = browser.find_element("id", "l_Dd-1-16")
Report_btn_el.click()

# Clicking the Generate Report button

In [11]:
time.sleep(1.5)
Generate_Report_btn_el = browser.find_element("id", "l_Dc-1-4")
Generate_Report_btn_el.click()

# Searching Fuel reports

In [12]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

input_field_el = browser.find_element("id", "Dc-h")
input_field_el.send_keys("Fuels report")
input_field_el.send_keys(Keys.ENTER)
    

## Fuels Report (Ethanol Section)

### Clicking thr Generate Report button

In [13]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-73")
Generate_btn_el.click()

### Defining Begining_RA

In [14]:
now = dt.datetime.now()

begining_month_closing_day = 10
begining_month = now.month -1
year = now.year
begining_month_closing = dt.datetime (year, begining_month, begining_month_closing_day)
print(begining_month_closing)

ONE_DAY = dt.timedelta(days=1)
HOLIDAYS_US = holidays.US()

def next_business_day():
    next_day = begining_month_closing + ONE_DAY
    while next_day.weekday() in holidays.WEEKEND or next_day in HOLIDAYS_US:
        next_day += ONE_DAY
    return next_day;

Begin_RA = next_business_day()
Begin_RA = Begin_RA.strftime("%d-%b-%Y")
print(Begin_RA)

2023-10-10 00:00:00
11-Oct-2023


### Defining End_RA

In [15]:
ending_month_closing_day = 10
ending_month = now.month 

ending_month_closing = dt.datetime (year, ending_month, ending_month_closing_day)
print(ending_month_closing)

def next_business_day():
    next_day = ending_month_closing + ONE_DAY
    while next_day.weekday() in holidays.WEEKEND or next_day in HOLIDAYS_US:
        next_day += ONE_DAY
    return next_day;

End_RA = next_business_day()
End_RA = End_RA.strftime("%d-%b-%Y")
print(End_RA)

2023-11-10 00:00:00
13-Nov-2023


### Inputing dates

In [16]:
time.sleep(1)
Begin_RA_el = browser.find_element("id", "Dg-2")
Begin_RA_el.send_keys(Begin_RA)
Begin_RA_el.send_keys(Keys.ENTER)

In [17]:
time.sleep(1)
End_RA_el = browser.find_element("id", "Dg-3")
End_RA_el.send_keys(End_RA)
End_RA_el.send_keys(Keys.ENTER)

### Clicking Generate button

In [18]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()
time.sleep(5)

### Clicking Export to excel button

In [19]:
Export_to_excel_el = browser.find_element("id", "Dc-9")
Export_to_excel_el.click()
time.sleep(5)

In [20]:
folder_path = "C:/Users/asifr/Downloads"
files_path = os.path.join(folder_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True) 
last_file = (files[0]) #latest file 
# print (files[0],files[1])#latest two files

In [21]:
# file name with extension
file_name = os.path.basename(last_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [22]:
file_name

'895653888'

In [23]:
import shutil
old_file = os.path.join(r"C:\Users\asifr\Downloads", (file_name + ".xls"))
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, ("eth" + " " +Begin_RA + " " + "to" + " " + End_RA + ".xls"))

try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file);


### Creating email attachment

In [24]:
fuel_reprt_email_item_1 = new_file

### Close the report

In [25]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")


for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: Fuels Report (Ethanol Section)' in button.get_attribute('title'):
        button.click();
        

## Fuels Report (Gasoline Section)

### Clicking thr Generate Report button

In [26]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-36")
Generate_btn_el.click()

### Inputing dates

In [27]:
time.sleep(1)
Begin_RA_el = browser.find_element("id", "Dg-2")
Begin_RA_el.send_keys(Begin_RA)
Begin_RA_el.send_keys(Keys.ENTER)

In [28]:
time.sleep(1)
End_RA_el = browser.find_element("id", "Dg-3")
End_RA_el.send_keys(End_RA)
End_RA_el.send_keys(Keys.ENTER)

### Clicking Generate button

In [29]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()
time.sleep(5)

### Clicking Export to excel button

In [30]:
Export_to_excel_el = browser.find_element("id", "Dc-9")
Export_to_excel_el.click()
time.sleep(5)

In [31]:
folder_path = "C:/Users/asifr/Downloads"
files_path = os.path.join(folder_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True) 
last_file = (files[0]) #latest file 
# print (files[0],files[1])#latest two files

In [32]:
# file name with extension
file_name = os.path.basename(last_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [33]:
file_name

'1432393728'

In [34]:
old_file = os.path.join(r"C:\Users\asifr\Downloads", (file_name + ".xls"))
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, ("gas" + " " +Begin_RA + " " + "to" + " " + End_RA + ".xls"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file);


### Creating email attachment

In [35]:
fuel_reprt_email_item_2 = new_file

### Close the report

In [36]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")


for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: Fuels Report (Gasoline Section)' in button.get_attribute('title'):
        button.click();
        

## Fuels Report (Diesel Section)

### Clicking thr Generate Report button

In [37]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-156")
Generate_btn_el.click()

### Inputing dates

In [38]:
time.sleep(1)
Begin_RA_el = browser.find_element("id", "Dg-2")
Begin_RA_el.send_keys(Begin_RA)
Begin_RA_el.send_keys(Keys.ENTER)

In [39]:
time.sleep(1)
End_RA_el = browser.find_element("id", "Dg-3")
End_RA_el.send_keys(End_RA)
End_RA_el.send_keys(Keys.ENTER)

### Clicking Generate button

In [40]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()
time.sleep(5)

### Clicking Export to excel button

In [41]:
Export_to_excel_el = browser.find_element("id", "Dc-9")
Export_to_excel_el.click()
time.sleep(5)

In [42]:
folder_path = "C:/Users/asifr/Downloads"
files_path = os.path.join(folder_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True) 
last_file = (files[0]) #latest file 
# print (files[0],files[1])#latest two files

In [43]:
# file name with extension
file_name = os.path.basename(last_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [44]:
file_name

'895522816'

In [45]:
old_file = os.path.join(r"C:\Users\asifr\Downloads", (file_name + ".xls"))
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, ("spf" + " " +Begin_RA + " " + "to" + " " + End_RA + ".xls"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file);


### Creating email attachment

In [46]:
fuel_reprt_email_item_3 = new_file

### Close the report

In [47]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")


for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: Fuels Report (Diesel Section)' in button.get_attribute('title'):
        button.click();
        

## Fuels Report _CFT & I

### Clicking thr Generate Report button

In [48]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-99")
Generate_btn_el.click()

### Inputing dates

In [49]:
time.sleep(1)
Begin_RA_el = browser.find_element("id", "Dg-2")
Begin_RA_el.send_keys(Begin_RA)
Begin_RA_el.send_keys(Keys.ENTER)

In [50]:
time.sleep(1)
End_RA_el = browser.find_element("id", "Dg-3")
End_RA_el.send_keys(End_RA)
End_RA_el.send_keys(Keys.ENTER)

### Clicking Generate button

In [51]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()
time.sleep(20)

### Clicking Export to excel button

In [52]:
Export_to_excel_el = browser.find_element("id", "Dc-9")
Export_to_excel_el.click()
time.sleep(5)

In [53]:
folder_path = "C:/Users/asifr/Downloads"
files_path = os.path.join(folder_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True) 
last_file = (files[0]) #latest file 
# print (files[0],files[1])#latest two files

In [54]:
# file name with extension
file_name = os.path.basename(last_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [55]:
file_name

'1969264640'

In [56]:
old_file = os.path.join(r"C:\Users\asifr\Downloads", (file_name + ".xls"))
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, ("pi" + " " +Begin_RA + " " + "to" + " " + End_RA + ".xls"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file);


### Creating email attachment

In [57]:
fuel_reprt_email_item_4 = new_file

### Close the report

In [58]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")


for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: Fuels Report _CFT P &amp;  I' in button.get_attribute('title'):
        button.click();
        

# Searching Alternative Fuel Volumes reports

In [59]:
time.sleep(5)
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(5)

input_field_el = browser.find_element("id", "Dc-h")
input_field_el.send_keys("Alternative Fuel Volumes")
input_field_el.send_keys(Keys.ENTER)
    

## Clicking Generate button

In [60]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-52")
Generate_btn_el.click()

## Inputing dates

In [61]:
time.sleep(1)
Begin_RA_el = browser.find_element("id", "Dg-2")
Begin_RA_el.send_keys(Begin_RA)
Begin_RA_el.send_keys(Keys.ENTER)

In [62]:
time.sleep(1)
End_RA_el = browser.find_element("id", "Dg-3")
End_RA_el.send_keys(End_RA)
End_RA_el.send_keys(Keys.ENTER)

## Clicking Generate button

In [63]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()
time.sleep(10)

## Clicking Export to excel button

In [64]:
Export_to_excel_el = browser.find_element("id", "Dc-9")
Export_to_excel_el.click()
time.sleep(5)

In [65]:
folder_path = "C:/Users/asifr/Downloads"
files_path = os.path.join(folder_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True) 
last_file = (files[0]) #latest file 
# print (files[0],files[1])#latest two files

In [66]:
# file name with extension
file_name = os.path.basename(last_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [67]:
file_name

'224434176'

In [68]:
old_file = os.path.join(r"C:\Users\asifr\Downloads", (file_name + ".xls"))
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, ("alt" + " " +Begin_RA + " " + "to" + " " + End_RA + ".xls"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file);


## Creating email attachment

In [69]:
fuel_reprt_email_item_5 = new_file

## Close the report

In [70]:
browser.switch_to.window(browser.window_handles[0])
time.sleep(2)

close_button_el = browser.find_elements(By.XPATH, "//button")


for button in close_button_el:
    if button.get_attribute('title') and 'Alternative Fuel Volumes' in button.get_attribute('title'):
        button.click();

In [71]:
time.sleep(1.5)
close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Group: Generate Report' in button.get_attribute('title'):
        button.click();

time.sleep(1.5)
Generate_Report_btn_el = browser.find_element("id", "l_Dc-1-4")
Generate_Report_btn_el.click()

# Searching CFT Locations reports

In [72]:
browser.switch_to.window(browser.window_handles[0])
time.sleep(1.5)

input_field_el = browser.find_element("id", "Dc-h")
input_field_el.send_keys("CFT Locations")
input_field_el.send_keys(Keys.ENTER)


## Clicking Generate button

In [73]:
time.sleep(3)

try:
  Generate_btn_el = browser.find_element("id", "l_Dc-6-365")
  Generate_btn_el.click()
except NoSuchElementException:
  Generate_btn_el = browser.find_element("id", "l_Dc-6-366")
  Generate_btn_el.click();
        
# Generate_btn_el = browser.find_element("id", "l_Dc-6-365")
# Generate_btn_el.click()

## Defining Begining_RA

In [74]:
now = dt.datetime.now()

begining_month_closing_day = 1
begining_month = now.month -2
year = now.year
begining_month_closing = dt.datetime (year, begining_month, begining_month_closing_day)

Begin_RA = begining_month_closing.strftime("%d-%b-%Y")
print(Begin_RA)

01-Sep-2023


## Defining End_RA

In [75]:
input_dt = dt.datetime.now()
print("The original date is:", input_dt.date())

from datetime import datetime, timedelta

last_month = input_dt.replace(day=28) - timedelta(days=32)
res = last_month - timedelta(days=last_month.day)
End_RA = res.strftime("%d-%b-%Y")
print(End_RA)

The original date is: 2023-11-22
30-Sep-2023


## Inputing dates

In [76]:
time.sleep(1)
Begin_RA_el = browser.find_element("id", "Dk-7")
Begin_RA_el.send_keys(Begin_RA)
Begin_RA_el.send_keys(Keys.ENTER)

In [77]:
time.sleep(1)
End_RA_el = browser.find_element("id", "Dk-8")
End_RA_el.send_keys(End_RA)
End_RA_el.send_keys(Keys.ENTER)
time.sleep(1)

In [78]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")
for button in close_button_el:
    if button.get_attribute('class') and 'FastMessageBoxButtonYes ui-button ui-corner-all ui-widget' in button.get_attribute('class'):
        button.click();

## Clicking Cube button

In [79]:
browser.switch_to.window(browser.window_handles[0])
time.sleep(5)

Cube_btn_el = browser.find_element("id", "cl_Dh-a")
Cube_btn_el.click()


## Clicking radio button

In [80]:
time.sleep(7)

radio_btn_el = browser.find_element("id", "cl_Df-g")
radio_btn_el.click();

## Clicking Spreadsheet button

In [81]:
time.sleep(5)

Spreadsheet_btn_el = browser.find_element("id", "l_Df_1-l-3")
Spreadsheet_btn_el.click()

In [82]:
folder_path = "C:/Users/asifr/Downloads"
files_path = os.path.join(folder_path, '*')
files = sorted(glob.iglob(files_path), key=os.path.getctime, reverse=True) 
last_file = (files[0]) #latest file 
# print (files[0],files[1])#latest two files

In [83]:
# file name with extension
file_name = os.path.basename(last_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [84]:
file_name

'848336896'

In [85]:
browser.implicitly_wait(3)
old_file = os.path.join(r"C:\Users\asifr\Downloads", (file_name + ".xls"))
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, ("Gasoline Gallons by Locations" + " " + End_RA + ".xls"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file);


## Creating email attachment

In [86]:
fuel_reprt_email_item_6 = new_file

## Close the report

In [87]:
browser.switch_to.window(browser.window_handles[0])
time.sleep(2)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: CFT Locations' in button.get_attribute('title'):
        button.click();

# Working on GL Backend RA - Main report

## Searching GL Backend RA - Main report

In [88]:
browser.switch_to.window(browser.window_handles[0])
time.sleep(3)

input_field_el = browser.find_element("id", "Dc-h")
input_field_el.send_keys("GL Backend RA - Main")
input_field_el.send_keys(Keys.ENTER)


## Clicking Generate button

In [89]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

# CIT

## Selecting CIT

In [90]:
time.sleep(1.5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("CIT")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [91]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()


## Clicking the View Report button

In [92]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'IC_Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Loading took too much time!


## Printing report

In [93]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("cit")
autoit.send("{Enter}")
time.sleep(5)
autoit.send("!{F4}")
time.sleep(3)
browser.close()




## Close the report

In [94]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")


for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        
    

## Renaming and moving the report

In [95]:
today = dt.date.today()
first = today.replace(day=1)
accrue_month_1 = first - dt.timedelta(days=25)
accrue_month_2 = first - dt.timedelta(days=32)
print(accrue_month_1.strftime("%Y-%b"))
print(accrue_month_2.strftime("%Y-%b"))

2023-Oct
2023-Sep


In [96]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "cit.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("CIT-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [97]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [98]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [99]:
cit_email_item_1 = new_file

# OGP-R

## Clicking Generate button

In [100]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting OGP-R

In [101]:
time.sleep(3)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("OGP-R")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [102]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [103]:
delay = 3 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [104]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("ogp-r")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [105]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [106]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "ogp-r.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("OGP-R-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [107]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [108]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [109]:
ogp_r_email_item_1 = new_file

# PIT

## Clicking Generate button

In [110]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting PIT

In [111]:
time.sleep(3)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("PIT")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [112]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [113]:
delay = 3 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [114]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("pit")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [115]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [116]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "pit.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("PIT-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [117]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [118]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [119]:
pit_email_item_1 = new_file

# PTW-O

## Clicking Generate button

In [120]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting PTW-O

In [121]:
time.sleep(3)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("PTW-O")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [122]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [123]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [124]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("ptw-o")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [125]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [126]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "ptw-o.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("PTW-O-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [127]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [128]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [129]:
ptw_o_email_item_1 = new_file

# PTW-R

## Clicking Generate button

In [130]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting PTW-R

In [131]:
time.sleep(3)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("PTW-R")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [132]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [133]:
delay = 3 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [134]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("ptw-r")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [135]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [136]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "ptw-r.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("PTW-R-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [137]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [138]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [139]:
ptw_r_email_item_1 = new_file

# WWT

## Clicking Generate button

In [140]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting WWT

In [141]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("WWT")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [142]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [143]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [144]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("wwt")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [145]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [146]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "wwt.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("WWT-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [147]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [148]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [149]:
wwt_email_item_1 = new_file

# FID

## Clicking Generate button

In [150]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting FID

In [151]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("FID")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [152]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [153]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [154]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("fid")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [155]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [156]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "fid.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("FID-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [157]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [158]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [159]:
fid_email_item_1 = new_file

# NWT

## Clicking Generate button

In [160]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting NWT

In [161]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("NWT")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [162]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [163]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [164]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("nwt")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [165]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [166]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "nwt.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("NWT-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [167]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [168]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [169]:
nwt_email_item_1 = new_file

# OGP

## Clicking Generate button

In [170]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting OGP

In [171]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("OGP")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [172]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [173]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [174]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("ogp")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [175]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [176]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "ogp.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("OGP-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [177]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [178]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [179]:
ogp_email_item_1 = new_file

# OGP-O

## Clicking Generate button

In [180]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting OGP-O

In [181]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("OGP-O")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [182]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [183]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [184]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("ogp-o")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [185]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [186]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "ogp-o.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("OGP-O-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [187]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [188]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [189]:
ogp_o_email_item_1 = new_file

# CAN

## Clicking Generate button

In [190]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting CAN

In [191]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("CAN")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [192]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [193]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [194]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("can")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [195]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [196]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "can.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("CAN-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [197]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [198]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [199]:
can_email_item_1 = new_file

# GRT

## Clicking Generate button

In [200]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting GRT

In [201]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("GRT")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [202]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [203]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [204]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("grt")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [205]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [206]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "grt.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("GRT-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [207]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [208]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [209]:
grt_email_item_1 = new_file

# CMP

## Clicking Generate button

In [210]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting CMP

In [211]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("CMP")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [212]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [213]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [214]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("cmp")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [215]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [216]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "cmp.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("CMP-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [217]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [218]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [219]:
cmp_email_item_1 = new_file

# ISI

## Clicking Generate button

In [220]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting ISI

In [221]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("ISI")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [222]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [223]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [224]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("isi")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [225]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [226]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "isi.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("ISI-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [227]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [228]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [229]:
isi_email_item_1 = new_file

# IPP

## Clicking Generate button

In [230]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting IPP

In [231]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("IPP")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [232]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [233]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [234]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("ipp")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [235]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [236]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "ipp.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("IPP-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [237]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [238]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [239]:
ipp_email_item_1 = new_file

# IPS

## Clicking Generate button

In [240]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting IPS

In [241]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("IPS")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [242]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [243]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [244]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("ips")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [245]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [246]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "ips.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("IPS-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [247]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [248]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [249]:
ips_email_item_1 = new_file

# IPH

## Clicking Generate button

In [250]:
time.sleep(1.5)
Generate_btn_el = browser.find_element("id", "l_Dc-6-166")
Generate_btn_el.click()

## Selecting IPH

In [251]:
time.sleep(5)
Revenue_Group_el = browser.find_element("name", "Dg-2")
Revenue_Group_el.send_keys("IPH")
Revenue_Group_el.send_keys(Keys.ENTER)

## Clicking the Generate button

In [252]:
Generate_btn_el = browser.find_element("id", "caption2_Dc-7")
Generate_btn_el.click()

## Clicking the View Report button

In [253]:
delay = 10 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'Dc-8')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")
time.sleep(10)     
View_Report_el = browser.find_element("id", "Dc-8")

View_Report_el.click()
time.sleep(1.5)

Page is ready!


## Printing report

In [254]:
browser.switch_to.window(browser.window_handles[1])
time.sleep(3)
autoit.send("^p")
time.sleep(3) # Pause to allow the PDF printer to load
autoit.send("{Enter}")
time.sleep(3) # Pause to allow the printer to print.
autoit.send("iph")
autoit.send("{Enter}")
time.sleep(3)
autoit.send("!{F4}")
time.sleep(3)
browser.close()

## Close the report

In [255]:
browser.switch_to.window(browser.window_handles[0])
browser.implicitly_wait(1.5)

close_button_el = browser.find_elements(By.XPATH, "//button")

for button in close_button_el:
    if button.get_attribute('title') and 'Close Manager: GL Backend RA - Main' in button.get_attribute('title'):
        button.click();
        

## Renaming and moving the report

In [256]:
old_file = os.path.join(r"C:\Users\asifr\OneDrive - State of New Mexico\Desktop", "iph.pdf")
old_file = shutil.move(old_file, destination_folder)
new_file = os.path.join(destination_folder, accrue_month_1.strftime("IPH-Prelim. %b %Y Reporting Generic Distribution Report.pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

In [257]:
# file name with extension
file_name = os.path.basename(new_file)

# file name without extension
file_name = (os.path.splitext(file_name)[0])

In [258]:
old_file = os.path.join(destination_folder, new_file)
new_file = os.path.join(destination_folder, accrue_month_2.strftime(file_name + ' ' + "(%b %Y Accrual).pdf"))
try:
  os.rename(old_file, new_file)
except FileExistsError:
  os.remove(new_file)
  os.rename(old_file, new_file)

## Creating Email attachment

In [259]:
iph_email_item_1 = new_file

In [260]:
browser.close()